## Python Visualization

### Data Analysis

In [130]:
# Imports
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [131]:
# Import data
data=pd.read_csv("Data\\LasVegasTripAdvisorReviews-Dataset.csv")
data.info()

data["Hotel_name"].nunique() # 21 hotels.
data["Review_month"].nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   User_country             504 non-null    object
 1   Number_of_reviews        504 non-null    int64 
 2   Number_of_hotel_reviews  504 non-null    int64 
 3   Helpful_votes            504 non-null    int64 
 4   Score                    504 non-null    int64 
 5   Period_of_stay           504 non-null    object
 6   Traveler_type            504 non-null    object
 7   Pool                     504 non-null    object
 8   Gym                      504 non-null    object
 9   Tennis_court             504 non-null    object
 10  Spa                      504 non-null    object
 11  Casino                   504 non-null    object
 12  Free_internet            504 non-null    object
 13  Hotel_name               504 non-null    object
 14  Hotel_stars              504 non-null    i

12

In [132]:
data.head()
data['User_country'].value_counts()
data['Hotel_stars'].unique()

array([3, 4, 5], dtype=int64)

### Visualization

In [133]:
# Make dataframe pipeline interactive
df = data.interactive(width=2200) 

In [134]:
# Select widget for Hotel stars
hote = data['Hotel_stars'].unique()
hotel_stars = pn.widgets.Select(name='Hotel stars', options=[3,4,5])
hotel_stars

Select(name='Hotel stars', options=[3, 4, 5], value=3)

In [135]:
# Radio buttons for Score and number of hotel reviews
yaxis_stars = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Score','Number_of_hotel_reviews'],
    button_type='success'
)
yaxis_stars

RadioButtonGroup(button_type='success', name='Y axis', options=['Score', 'Number_of_hotel...], value='Score')

In [136]:
# Need to connect data pipeline to widgets so that they are getting updated in real time.
hotels_stars_pipeline = (
    df[
        (df.Hotel_stars == hotel_stars)
    ]
     .groupby(['Hotel_name'])[yaxis_stars].mean()
     .to_frame()
     .reset_index()  
     .reset_index(drop=True)
 )
hotels_stars_pipeline

In [137]:
# Score based plot
hotels_stars_plot = hotels_stars_pipeline.hvplot(x = 'Hotel_name', y=yaxis_stars, line_width=2, title="Score of Hotels based on Hotel stars",width=500, height=300)
hotels_stars_plot

In [138]:
# Select widget for hotel names
ho_names = data['Hotel_name'].unique()
hotels = pn.widgets.Select(name='Hotels', options=['Circus Circus Hotel & Casino Las Vegas',
       'Excalibur Hotel & Casino', 'Monte Carlo Resort&Casino',
       'Treasure Island- TI Hotel & Casino',
       'Tropicana Las Vegas - A Double Tree by Hilton Hotel',
       'Caesars Palace', 'The Cosmopolitan Las Vegas',
       'The Palazzo Resort Hotel Casino', 'Wynn Las Vegas',
       'Trump International Hotel Las Vegas', 'The Cromwell',
       'Encore at wynn Las Vegas',
       'Hilton Grand Vacations on the Boulevard',
       "Marriott's Grand Chateau", 'Tuscany Las Vegas Suites & Casino',
       'Hilton Grand Vacations at the Flamingo', 'Wyndham Grand Desert',
       'The Venetian Las Vegas Hotel', 'Bellagio Las Vegas',
       'Paris Las Vegas', 'The Westin las Vegas Hotel Casino & Spa'])

In [139]:
# Radio buttons for Score
yaxis_Score = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Score'],
    button_type='success'
)
yaxis_stars

# Connect data pipeline to widgets.

hotels_pipeline_features = (
    df[
        (df.Hotel_name == hotels)
    ]
    .groupby(['Hotel_name','Hotel_stars','Number_of_rooms','Tennis_court','Pool','Gym','Spa','Casino','Free_internet'])[yaxis_Score].mean()
     .to_frame()
     .reset_index()  
     .reset_index(drop=True)
 )
hotels_pipeline_features

hotels_table = hotels_pipeline_features.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
hotels_table

In [140]:
# Select widget for User_country

ho_stars = data['Hotel_stars'].unique()
hotel_stars = pn.widgets.Select(name='Hotel stars', options=[3,4,5])

y_axis= data['User_country'].value_counts()

hotels_country_pipeline = (
    df[
        (df.Hotel_name == hotels)
    ]
       .groupby(['User_country']).count()
       .reset_index()
       .sort_values(by='User_country')  
       .reset_index(drop=True)
 )
hotels_country_pipeline

hotels_country_pipeline_bar = hotels_country_pipeline.hvplot(kind='bar', 
                                                     x='User_country', 
                                                     y='Number_of_rooms', 
                                                     title='Hotel usage based on Country',ylabel='Usage')
hotels_country_pipeline_bar

In [141]:
# Dashboard
template = pn.template.FastListTemplate(
    title='Las Vegas Hotels', 
    sidebar=[pn.pane.Markdown("# Las Vegas Hotels"), 
             pn.pane.Markdown("#### This dashboard is used to know the details of the Las Vegas Hotels."), 
             pn.pane.JPG("https://th.bing.com/th/id/OIP.4aa3iz122v2B_TYGDPgDtAHaHa?pid=ImgDet&rs=1", sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings")   
             ,hotels],
    
    main=[pn.Row(pn.Column(hotel_stars,yaxis_stars,hotels_stars_plot.panel(width=700), margin=(0,25))), 
          pn.Row(pn.Column(hotels_table.panel(width=500))),
          pn.Row(pn.Column(hotels_country_pipeline_bar.panel(width=600)))],
    accent_base_color="#2F4F4F",
    header_background="#A01346",
)
template.show()
#template.servable();

Launching server at http://localhost:51489
